In [1]:
import numpy as np

# Activation Functions
def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - np.square(np.tanh(x))

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def d_sigmoid(x):
    return (1 - sigmoid(x)) * sigmoid(x)

# Loss Functions
def logloss(y, a):
    return -(y*np.log(a) + (1-y)*np.log(1-a))

def d_logloss(y, a):
    return (a - y)/(a*(1 - a))


# The layer class
class Layer:

    activationFunctions = {
        'tanh': (tanh, d_tanh),
        'sigmoid': (sigmoid, d_sigmoid)
    }
    learning_rate = 0.1

    def __init__(self, inputs, neurons, activation):
        self.W = np.random.randn(neurons, inputs)
        self.b = np.zeros((neurons, 1))
        self.act, self.d_act = self.activationFunctions.get(activation)

    def feedforward(self, A_prev):
        self.A_prev = A_prev
        self.Z = np.dot(self.W, self.A_prev) + self.b
        self.A = self.act(self.Z)
        return self.A

    def backprop(self, dA):
        dZ = np.multiply(self.d_act(self.Z), dA)
        dW = 1/dZ.shape[1] * np.dot(dZ, self.A_prev.T)
        db = 1/dZ.shape[1] * np.sum(dZ, axis=1, keepdims=True)
        dA_prev = np.dot(self.W.T, dZ)

        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db

        return dA_prev

In [42]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse=False)

iris = load_iris()

x_train = iris.data # dim x m
y_train = one_hot_encoder.fit_transform(np.array(iris.target).reshape(-1, 1)).T # 1 x m

print(y_train.shape)

m = y_train.shape[1]
epochs = 1500

layers = [Layer(150, 4, 'tanh'), Layer(4, 3, 'sigmoid')]
costs = [] # to plot graph

for epoch in range(epochs):
    # Feedforward
    A = x_train
    for layer in layers:
        A = layer.feedforward(A)
        print(A.shape)

    print(A)

    # Calulate cost to plot graph
    cost = 1/m * np.sum(logloss(y_train, A))
    costs.append(cost)

    # Backpropagation
    dA = d_logloss(y_train, A)
    for layer in reversed(layers):
        dA = layer.backprop(dA)


# Making predictions
A = x_train
for layer in layers:
    A = layer.feedforward(A)
print(A)

(3, 150)
(4, 4)
(3, 4)
[[0.25264852 0.65045728 0.54603446 0.54418939]
 [0.57706673 0.49484481 0.18681197 0.18646526]
 [0.4237063  0.32046632 0.6103581  0.60619166]]


ValueError: operands could not be broadcast together with shapes (3,150) (3,4) 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(epochs), costs)